In [1]:
import pandas as pd
import time
from multiprocessing import Pool
from sklearn.model_selection import ParameterGrid
import csv

In [2]:
def preProcessData(stockName, ST, LT):
    df = pd.read_csv("IntradayData_2018/"+stockName+".txt")
    df.columns = ['stock', 'date','time','openingPrice','high','low','closingPrice','volume','e']
    df = df.drop('e', 1)
    for i in ST:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
    for i in LT:
        df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
    dateList = list(set(df.date.values))
    dateList = sorted(dateList, key=abs, reverse=False)
    return df, dateList

In [3]:
def convertTime(timeString):
    timeArr = timeString.split(':')
    time = int(timeArr[0])*60 + int(timeArr[1])
    return time

def revConvertTime(timeInt):
    timeHr = timeInt/60
    timeMin = timeInt%60
    return str(timeHr) + ':' + str(timeMin)

def exitTrade(buyPrice, sellPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice(1-0.0002) - buyPrice)/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def parseRow(row, ST, LT):
    time = convertTime(row[1].time)
    openingPrice = row[1].openingPrice
    low = row[1].low
    high = row[1].high
    closingPrice = row[1].closingPrice
    listPrice = 0.5*(openingPrice + closingPrice)
    diff = row[1]["MA_"+str(LT)] - row[1]["MA_"+str(ST)]
    return time, openingPrice, low, high, closingPrice, listPrice, diff

In [4]:
def dayScript(df, ST, LT, money, targetPercentage, stopLossPercentage, entryDifference, verbose = False):
    eomDays = 0
    boughtFlag = 0
    dailyTrades = 0
    buyPrice = 0
    sellPrice = 0
    sellMargin = 1 + targetPercentage
    buyMargin = 1 - targetPercentage

    for row in df.iterrows():
        time, openingPrice, low, high, closingPrice, listPrice, diff = parseRow(row, ST, LT)       
        if(time >= 570 and time <900):
            if(boughtFlag == 0):
                #market enter logic
                if(diff>0 and diff<entryDifference*listPrice):
                    #buy
                    buyPrice = listPrice
                    sellPrice = 0
                    timeEntered = time
                    boughtFlag = 1
                    stopLossPrice = buyPrice * (1 - stopLossPercentage)
                    stopLossWindow = buyPrice*stopLossPercentage
                elif(diff<0 and diff>-1*entryDifference*listPrice):
                    #sell
                    sellPrice = listPrice
                    buyPrice = 0
                    timeEntered = time
                    boughtFlag = -1
                    stopGainPrice = sellPrice * (1 + stopLossPercentage)
                    stopGainWindow = sellPrice*stopLossPercentage
            elif(boughtFlag==1):
                #market exit logic (when already bought)
                if(high > buyPrice*sellMargin):
                    percentageProfit = exitTrade(buyPrice, buyPrice*sellMargin, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('target exit')
                elif(stopLossPrice > low):
                    percentageProfit = exitTrade(buyPrice, stopLossPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('stopLoss exit')
            else:
                #market exit logic (when already sold)
                if(sellPrice*buyMargin > low):
                    percentageProfit = exitTrade(sellPrice*buyMargin, sellPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('target exit')
                elif(high > stopGainPrice):
                    percentageProfit = exitTrade(stopGainPrice, sellPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('stopLoss exit')
            #trailing stopLoss stopGain calculations.
            if(boughtFlag ==1 ):
                trail = (listPrice - stopLossPrice)
                if(trail > stopLossWindow ):
                    climb = trail - stopLossWindow
                    stopLossPrice+= climb
                    
            elif(boughtFlag ==-1 ):
                trail = (stopGainPrice - listPrice)
                if(trail > stopGainWindow ):
                    climb = trail - stopGainWindow
                    stopGainPrice-= climb
                
        elif(time >= 900):
            if(boughtFlag == 1):
                percentageProfit =exitTrade(buyPrice, closingPrice, timeEntered, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1
            elif(boughtFlag == -1):
                percentageProfit = exitTrade(closingPrice, sellPrice, timeEntered, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1

#     print('Total Profit : ',totalProfit)
    return money, dailyTrades, eomDays

In [5]:
def movingAverageExperiment(paramList):
    df, dateList, ST,LT,targetPercentage,stopLossPercentage,entryDifference = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        updatedMoney, dailyTrades, eomDays = dayScript(new_df, ST, LT,  money, targetPercentage, stopLossPercentage, entryDifference, verbose = False)
#         print(date, money, updatedMoney, dailyTrades)
        money = updatedMoney

#     print('Eom days - ',eomDays)
    yearlyProfitPercentage = (money - 1) * 100
    print(ST,LT,targetPercentage,stopLossPercentage,entryDifference, 'Total Profit %= ', yearlyProfitPercentage)
   
    return yearlyProfitPercentage

In [6]:
def bruteAnalysis(stockName, paramList, pool, threadPoolSize):
    param_grid = {'ST' : ST, 'LT': LT, 'targetPercentage': targetPercentage, 'stopLossPercentage': stopLossPercentage, 'entryDifference':entryDifference}
    grid = ParameterGrid(param_grid)

    csvList = []
    df, dateList = preProcessData(stockName, ST, LT)
    i=0
    print('param combination = ' ,len(grid))
    while i<len(grid):
        param_list =[]
        for j in range(threadPoolSize):
            if(i+j < len(grid)):
                params = grid[i+j]
                param_list.append([df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference']])
        start = time.time()
        resultList = pool.map(movingAverageExperiment, param_list)
        end = time.time()
        for j in range(len(param_list)):
            params = param_list[j]
            result = resultList[j]
            new_point = {'StockName':stockName, 
                         'ST':params[2], 
                         'LT':params[3], 
                         'targetPercentage':params[4], 
                         'stopLossPercentage':params[5],
                         'entryDifference':params[6], 
                         'profitPercentage':result,
                        }
            csvList.append(new_point)
        print('Time - ',end - start)
        i+=threadPoolSize
        print(i,len(grid))
    return csvList

In [7]:
# def writeCsvHeader():
#     f = csv.writer(open("result.csv", "wb+"))
#     f.writerow(["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])
#     return f
# def writeToCsv(csvList):
#     f.writerow(["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])
#     for x in csvList:
#         f.writerow([x["stockName"],
#                     x['params'][2],
#                     x['params'][3],
#                     x['params'][4],
#                     x['params'][5],
#                     x['params'][6],
#                     x["result"]])

In [12]:
stockList = ['ACC_F1','ASHOKLEY_F1','AXISBANK_F1','BHARTIARTL_F1','RELIANCE_F1','INFY_F1','WIPRO_F1','PNB_F1','SBIN_F1','SUNPHARMA_F1','GRASIM_F1','LUPIN_F1','LT_F1','HINDUNILVR_F1']

In [13]:
# #old
# ST = [1,4,12,20]
# LT = [100,200,500,1000]
# targetPercentage = [0.01, 0.05]
# stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
# entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
# # #new
# ST = [10,15,20,25]
# LT = [75,100,200,500]
# targetPercentage = [0.01, 0.05, 0.075]
# stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
# entryDifference = [0.00005, 0.000075, 0.0001]
# # #v3
ST = [15,20]
LT = [100]
targetPercentage = [0.05]
stopLossPercentage = [0.007]
entryDifference = [0.00005, 0.000075]

paramList = [LT, ST, targetPercentage, stopLossPercentage, entryDifference]

In [14]:
# f = csv.writer(open("result.csv", "wb+"))
# f.writerow(["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

threadPoolSize = 8
pool = Pool(threadPoolSize)
with open('result.csv', 'w') as f_out:
    out_colnames = ["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"]
    csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    csv_writer.writeheader()
    for stock in stockList:
        csvList = bruteAnalysis(stock[:-3], paramList,pool,threadPoolSize)
        for point in csvList:
            csv_writer.writerow(point)
            
pool.terminate()
pool.join()

('param combination = ', 4)
(15, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', -15.525793348020967)
(20, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', -24.44694856813583)
(15, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', 3.2407335151020122)
(20, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', -19.690621243638972)
('Time - ', 23.416100025177002)
(8, 4)
('param combination = ', 4)
(15, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', 0.14050295355858555)
(15, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', -0.9250390616260651)
(20, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', 40.21121727223702)
(20, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', 17.22839509708407)
('Time - ', 27.70678687095642)
(8, 4)
('param combination = ', 4)
(15, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', 5.567727983495985)
(15, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', -5.705881935574297)
(20, 100, 0.05, 0.007, 7.5e-05, 'Total Profit %= ', -5.62699814923564)
(20, 100, 0.05, 0.007, 5e-05, 'Total Profit %= ', -5.2775